# 对活动数据进行分析
（只取训练集和测试集中出现的样本）

数据来源于Kaggle竞赛：Event Recommendation Engine Challenge，根据
    events they’ve responded to in the past
    user demographic information
    what events they’ve seen and clicked on in our app
预测用户对某个活动是否感兴趣

竞赛官网：
https://www.kaggle.com/c/event-recommendation-engine-challenge/data


活动描述信息在events.csv文件：共110维特征
前9列：event_id, user_id, start_time, city, state, zip, country, lat, and lng.
event_id：id of the event, 
user_id：id of the user who created the event.  
city, state, zip, and country： more details about the location of the venue (if known).
lat and lng： floats（latitude and longitude coordinates of the venue）
start_time： 字符串，ISO-8601 UTC time，表示活动开始时间

后101列为词频：count_1, count_2, ..., count_100，count_other
count_N：活动描述出现第N个词的次数
count_other：除了最常用的100个词之外的其余词出现的次数

这里我们用count_1, count_2, ..., count_100，count_other属性做聚类，即活动用这些关键词来描述，可表示活动的类别

# 导入工具包

In [1]:
#数据量太大，pdandas不能一次讲所有数据读入
#也可以用pandas,一次读取部分数据，可以参考：https://www.cnblogs.com/datablog/p/6127000.html
#import pandas as pd

import numpy as np
import scipy.sparse as ss
import scipy.io as sio

#保存数据
import pickle

#event的特征需要编码
from utils import FeatureEng
from sklearn.preprocessing import normalize
#相似度/距离
import scipy.spatial.distance as ssd

# 统计活动数目

In [2]:
#读取数据，并统计有多少不同的events
#其实EDA.ipynb中用read_csv已经统计过了
lines = 0
fin = open("events.csv", 'r')
#找到用C/C++的感觉了
#字段：event_id, user_id,start_time, city, state, zip, country, lat, and lng， 101 columns of words count
fin.readline() # skip header，列名行
for line in fin:
    cols = line.strip().split(",")
    lines += 1
fin.close()
print(cols)

print("number of records :%d" % lines)

['1373240562', '3334738518', '2013-02-14T20:00:00.002Z', '', '', '', '', '', '', '0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', ' 0', '1']
number of records :3137972


活动数目太多（300w+），训练+测试集的活动没这么多，所有先去处理train和test，得到竞赛需要用到的活动和用户
然后对在训练集和测试集中出现过的活动和用户建立新的ID索引
先运行user_event.ipynb,
得到活动列表文件：PE_eventIndex.pkl

# 读取之前算好的测试集和训练集中出现过的活动
详见user_event.ipynb

In [3]:
#读取训练集和测试集中出现过的活动列表
eventIndex = pickle.load(open("PE_eventIndex.pkl", 'rb'))
n_events = len(eventIndex)

print("number of events in train & test :%d" % n_events)

number of events in train & test :13418


# 处理events.csv --> 特征编码、活动之间的相似度

In [8]:
FE = FeatureEng()

fin = open("events.csv", 'r')

#字段：event_id, user_id,start_time, city, state, zip, country, lat, and lng， 101 columns of words count
fin.readline() # skip header

#start_time, city, state, zip, country, lat, and lng
eventPropMatrix = ss.dok_matrix((n_events, 7))

#词频特征
eventContMatrix = ss.dok_matrix((n_events, 101))

for line in fin.readlines():
    cols = line.strip().split(",")
    eventId = str(cols[0])
    
    #if eventIndex.has_key(eventId):  #在训练集或测试集中出现  python2 的语法
    if eventId in eventIndex:  #在训练集或测试集中出现
        i = eventIndex[eventId]
  
        #event的特征编码，这里只是简单处理，其实开始时间，地点等信息很重要
        eventPropMatrix[i, 0] = FE.getJoinedYearMonth(cols[2]) # start_time
        eventPropMatrix[i, 1] = FE.getFeatureHash(cols[3].encode("utf8")) # city
        eventPropMatrix[i, 2] = FE.getFeatureHash(cols[4].encode("utf8")) # state
        eventPropMatrix[i, 3] = FE.getFeatureHash(cols[5].encode("utf8")) # zip
        eventPropMatrix[i, 4] = FE.getFeatureHash(cols[6].encode("utf8")) # country
        eventPropMatrix[i, 5] = FE.getFloatValue(cols[7]) # lat
        eventPropMatrix[i, 6] = FE.getFloatValue(cols[8]) # lon
        
        #词频
        for j in range(9, 110):
            eventContMatrix[i, j-9] = cols[j]
fin.close()

#用L2模归一化
eventPropMatrix = normalize(eventPropMatrix,
    norm="l2", axis=0, copy=False)
sio.mmwrite("EV_eventPropMatrix", eventPropMatrix)

#词频，可以考虑我们用这部分特征进行聚类，得到活动的genre
eventContMatrix = normalize(eventContMatrix,
    norm="l2", axis=0, copy=False)
sio.mmwrite("EV_eventContMatrix", eventContMatrix)


# calculate similarity between event pairs based on the two matrices
eventPropSim = ss.dok_matrix((n_events, n_events))
eventContSim = ss.dok_matrix((n_events, n_events))

#读取在测试集和训练集中出现的活动对
uniqueEventPairs = pickle.load(open("PE_uniqueEventPairs.pkl", 'rb'))

for e1, e2 in uniqueEventPairs:
    #i = eventIndex[e1]
    #j = eventIndex[e2]
    i = e1
    j = e2
    
    #非词频特征，采用Person相关系数作为相似度
    #if not eventPropSim.has_key((i,j)):
    if not (i,j) in eventPropSim:
        epsim = ssd.correlation(eventPropMatrix.getrow(i).todense(),
            eventPropMatrix.getrow(j).todense())
        
        eventPropSim[i, j] = epsim
        eventPropSim[j, i] = epsim
    
    #对词频特征，采用余弦相似度，也可以用直方图交/Jacard相似度
    #if not eventContSim.has_key((i,j)):
    if not (i,j) in eventContSim:
        ecsim = ssd.cosine(eventContMatrix.getrow(i).todense(),
            eventContMatrix.getrow(j).todense())
    
        eventContSim[i, j] = epsim
        eventContSim[j, i] = epsim
    
sio.mmwrite("EV_eventPropSim", eventPropSim)
sio.mmwrite("EV_eventContSim", eventContSim)

In [9]:
eventPropSim.getrow(0).todense()

matrix([[0., 0., 0., ..., 0., 0., 0.]])

In [10]:
print(e1,e2)

7976 11674


In [11]:
print(eventPropSim)

  (9987, 11960)	0.5922860756125148
  (11960, 9987)	0.5922860756125148
  (9475, 8556)	1.0840521935022833
  (8556, 9475)	1.0840521935022833
  (10782, 5359)	0.8768488435616606
  (5359, 10782)	0.8768488435616606
  (4522, 8619)	1.0324888141036443
  (8619, 4522)	1.0324888141036443
  (2324, 7961)	0.8071839337158427
  (7961, 2324)	0.8071839337158427
  (11101, 8094)	0.0001434449671897653
  (8094, 11101)	0.0001434449671897653
  (10088, 12203)	0.9355518787338608
  (12203, 10088)	0.9355518787338608
  (12130, 1815)	1.01362186350775
  (1815, 12130)	1.01362186350775
  (5505, 5510)	0.9522807253464107
  (5510, 5505)	0.9522807253464107
  (8138, 9972)	0.9297215306496674
  (9972, 8138)	0.9297215306496674
  (9665, 4806)	1.7598145163333356e-12
  (4806, 9665)	1.7598145163333356e-12
  (4257, 3146)	1.0135374863373015
  (3146, 4257)	1.0135374863373015
  (9121, 7275)	0.3136994245946758
  :	:
  (1744, 2629)	0.008353706533611893
  (9109, 3035)	0.006625727610589216
  (3035, 9109)	0.006625727610589216
  (1973, 737)	

In [12]:
print(eventPropMatrix)

  (8255, 0)	0.008591382007347012
  (9156, 0)	0.008844069713445453
  (1058, 0)	0.008844069713445453
  (3929, 0)	0.008591382007347012
  (11114, 0)	0.00833869430124857
  (2512, 0)	0.008844069713445453
  (622, 0)	0.008844069713445453
  (9979, 0)	0.008591382007347012
  (12854, 0)	0.008591382007347012
  (9431, 0)	0.008591382007347012
  (7672, 0)	0.008844069713445453
  (10446, 0)	0.008086006595150129
  (5551, 0)	0.008844069713445453
  (934, 0)	0.009096757419543895
  (10038, 0)	0.008086006595150129
  (9952, 0)	0.008086006595150129
  (11397, 0)	0.008591382007347012
  (11561, 0)	0.008844069713445453
  (694, 0)	0.008844069713445453
  (12160, 0)	0.00833869430124857
  (12625, 0)	0.00833869430124857
  (4760, 0)	0.008591382007347012
  (12033, 0)	0.008844069713445453
  (8474, 0)	0.008591382007347012
  (8257, 0)	0.008591382007347012
  :	:
  (11559, 6)	0.00022266477771758144
  (10463, 6)	-0.00908034686010137
  (12068, 6)	-0.013845934738009607
  (9689, 6)	-0.014267160802005031
  (3366, 6)	0.0127810518467